## Displacement

In [1]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from model.Displacement.regression import CNN
from dataset.Displacement.DamageIdentification import DamageIdentificationDataset
from torch.utils.data import DataLoader

/home/tingyan/.conda/envs/icshm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mode = "test"
dataset = DamageIdentificationDataset(path="./Data", mode=mode)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)

In [3]:
scratch = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/From_scratch/version_2/checkpoints/epoch=00343-val_loss=0.0000.ckpt").eval().to("cuda")
transfer_AE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_AE/version_2/checkpoints/epoch=00034-val_loss=0.0197.ckpt").eval().to("cuda")
transfer_DamageAE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_DamageAE/version_2/checkpoints/epoch=00500-val_loss=0.0142.ckpt").eval().to("cuda")
transfer_TripletAE = CNN.load_from_checkpoint("./Logs/Identification/Displacement-regression/Transfer_TripletAE/version_2/checkpoints/epoch=00497-val_loss=0.0122.ckpt").eval().to("cuda")

In [4]:
scratch_prediction = []
AE_prediction = []
dAE_prediction = []
tAE_prediction = []

target_list = []
situation_list = []

for input, target, situation in dataloader:
    with torch.no_grad():
        input = input.to("cuda")       
        scratch_prediction += scratch(input).unsqueeze(0)
        AE_prediction += transfer_AE(input).unsqueeze(0)
        dAE_prediction += transfer_DamageAE(input).unsqueeze(0)
        tAE_prediction += transfer_TripletAE(input).unsqueeze(0)

        target_list += target.unsqueeze(0)
        situation_list += situation.unsqueeze(0)

scratch_prediction = torch.concat(scratch_prediction).cpu().numpy()
AE_prediction = torch.concat(AE_prediction).cpu().numpy()
dAE_prediction = torch.concat(dAE_prediction).cpu().numpy()
tAE_prediction = torch.concat(tAE_prediction).cpu().numpy()

target_list = torch.concat(target_list).cpu().numpy()
situation_list = torch.concat(situation_list).cpu().numpy()

In [11]:
from scipy.stats import norm

def draw_predition_distribution(pred, target_list, id, model_name, mode):
    index = situation_list == id
    plt.rcParams['text.usetex'] = True
    plt.rcParams['text.latex.preamble'] = r'\centering'
    pred = scratch_prediction[index]
    max_value, min_value = pred.max(), pred.min()
    pred_no7, pred_no22, pred_no38 = pred[:, 0], pred[:, 1], pred[:, 2]
    target_no7, target_no22, target_no38 = target_list[index][0][0], target_list[index][0][1], target_list[index][0][2]

    (mu_no7, sigma_no7) = norm.fit(pred_no7)
    (mu_no22, sigma_no22) = norm.fit(pred_no22)
    (mu_no38, sigma_no38) = norm.fit(pred_no38)

    fig, ax = plt.subplots(figsize=(6, 6))

    ax.hist(pred_no7, 10, density=True, range=(min_value, max_value), facecolor='tab:blue', alpha=0.5)
    ax.hist(pred_no22, 10, density=True, range=(min_value, max_value), facecolor='tab:orange', alpha=0.5)
    ax.hist(pred_no38, 10, density=True, range=(min_value, max_value), facecolor='tab:green', alpha=0.5)

    ax.axvline(x = target_no7, linestyle = "-", color = 'tab:blue', label = 'Target of No.7')
    ax.axvline(x = target_no22, linestyle = "--", color = 'tab:orange', label = 'Target of No.22')
    ax.axvline(x = target_no38, linestyle = "-.", color = 'tab:green', label = 'Target of No.38')

    xmin, xmax, ymin, ymax = plt.axis()
    x = np.linspace(xmin, xmax, 100)
    y = norm.pdf(x, mu_no7, sigma_no7)
    l = ax.plot(x, y, linestyle = 'dotted', color = "tab:blue")
    y = norm.pdf(x, mu_no22, sigma_no22)
    l = ax.plot(x, y, linestyle = 'dotted', color = "tab:orange")
    y = norm.pdf(x, mu_no38, sigma_no38)
    l = ax.plot(x, y, linestyle = 'dotted', color = "tab:green")
    ax.set_title(
        r'$\mu_{no.07}=%.3f$, $\sigma_{no.07}=%.3f;$'
        r'$\mu_{no.22}=%.3f$, $\sigma_{no.22}=%.3f;$\\' 
        r'$\mu_{no.38}=%.3f$, $\sigma_{no.38}=%.3f;$' % (mu_no7, sigma_no7, mu_no22, sigma_no22, mu_no38, sigma_no38))
    ax.set_xlabel("Reduction of elastic modulus")
    fig.suptitle(f"Scenario {id}", fontsize=16)
    fig.legend(ncol = 3, bbox_to_anchor=(0.92, 0.04))
    if not os.path.exists(f"./pic/distribution/{model_name}/{mode}"):
        os.makedirs(f"./pic/distribution/{model_name}/{mode}")
    plt.savefig(f"./pic/distribution/{model_name}/{mode}/Scenario{id:02d}.svg", bbox_inches='tight')
    plt.savefig(f"./pic/distribution/{model_name}/{mode}/Scenario{id:02d}.png", bbox_inches='tight')
    plt.close()

    if (mu_no7 - 3 * sigma_no7) <= target_no7 <= (mu_no7 + 3 * sigma_no7) and (mu_no22 - 3 * sigma_no22) <= target_no22 <= (mu_no22 + 3 * sigma_no22) and (mu_no38 - 3 * sigma_no38) <= target_no38 <= mu_no38 + 3 * sigma_no38:
        return True
    else:
        return False

In [12]:
scratch = []
AE = []
dAE = []
tAE = []
for index in range(1, 12):
    scratch.append(draw_predition_distribution(scratch_prediction, target_list, index, model_name="Scratch", mode=mode))
    AE.append(draw_predition_distribution(AE_prediction, target_list, index, model_name="Transfer(AE)", mode=mode))
    dAE.append(draw_predition_distribution(dAE_prediction, target_list, index, model_name="Transfer(DamageAE)", mode=mode))
    tAE.append(draw_predition_distribution(tAE_prediction, target_list, index, model_name="Transfer(TripletAE)", mode=mode))

In [13]:
print(scratch)
print(AE)
print(dAE)
print(tAE)

[True, False, True, True, True, True, True, True, False, True, True]
[True, False, True, True, True, True, True, True, False, True, True]
[True, False, True, True, True, True, True, True, False, True, True]
[True, False, True, True, True, True, True, True, False, True, True]
